In [68]:
from trackml.dataset import load_event
from trackml.dataset import load_dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random
from sklearn.preprocessing import LabelEncoder

def proc_arr(x):
    for i in range(len(x) - 1):
        block_stat.ix[x[i], x[i+1]] += 1
        
def proc_arr_next_hit(x):
    for i in range(len(x) - 1):
        next_hits.ix[x[i], 'next_hit'] = x[i+1]

In [69]:
from gensim.models import KeyedVectors
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten
from keras.layers.wrappers import TimeDistributed
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.preprocessing.sequence import pad_sequences
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers import Embedding, Dense, Dropout, Reshape, Merge, BatchNormalization, TimeDistributed, Lambda
from keras.regularizers import l2
from keras.callbacks import Callback, ModelCheckpoint
from keras import backend as K
from keras.layers.merge import concatenate
from keras.layers.wrappers import Bidirectional
import keras
from keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler


In [59]:
cnt = 0
X_train = pd.DataFrame()
X_val = pd.DataFrame()

for event_id, hits, cells, particles, truth in load_dataset('data/train_5.zip'):
        
    if cnt == 30:
        break
    
    truth_hits = pd.concat([hits, truth.drop(['hit_id'], axis=1)], axis=1)
    truth_hits['dist'] = truth_hits.apply(lambda x: x[1]**2 + x[2]**2 + x[3]**2, axis=1)
    truth_hits['rad_dist'] = truth_hits.apply(lambda x: x[1]**2 + x[2]**2, axis=1)
    
    truth_hits = truth_hits[truth_hits.particle_id != 0].sort_values(['particle_id', 'rad_dist'])
    truth_hits['block'] = truth_hits['volume_id'].astype(str) + "_" + truth_hits['layer_id'].astype(str)
    
    cell_stats = cells.groupby('hit_id')['value'].mean().reset_index().rename({'value':'value_mean'},axis=1)

    tmp = cells.groupby('hit_id')['value'].max().reset_index().rename({'value':'value_max'},axis=1)
    cell_stats = pd.merge(cell_stats, tmp, on='hit_id', how='left')

    tmp = cells.groupby('hit_id')['value'].min().reset_index().rename({'value':'value_min'},axis=1)
    cell_stats = pd.merge(cell_stats, tmp, on='hit_id', how='left')

    tmp = cells.groupby('hit_id')['value'].sum().reset_index().rename({'value':'value_sum'},axis=1)
    cell_stats = pd.merge(cell_stats, tmp, on='hit_id', how='left')

    tmp = cells.groupby('hit_id')['value'].std().reset_index().rename({'value':'value_std'},axis=1)
    cell_stats = pd.merge(cell_stats, tmp, on='hit_id', how='left')

    tmp = cells.groupby('hit_id')['value'].count().reset_index().rename({'value':'value_count'},axis=1)
    cell_stats = pd.merge(cell_stats, tmp, on='hit_id', how='left')

    truth_hits = pd.merge(truth_hits, cell_stats, on='hit_id', how='left')    
    
    
    next_hits = pd.DataFrame(truth_hits['hit_id'].unique()).rename({0:'hit_id'},axis=1)
    next_hits['next_hit'] = np.nan

    truth_hits['next_particle'] = list(truth_hits['particle_id'].values[1:]) + [-1]
    truth_hits['next_hit'] = list(truth_hits['hit_id'].values[1:]) + [-1]
    truth_hits['not_end_hit'] = truth_hits['particle_id'] == truth_hits['next_particle']
    truth_hits['next_hit']  = truth_hits['not_end_hit'] * truth_hits['next_hit']
    del truth_hits['next_particle']
#     truth_hits = pd.merge(truth_hits, truth_hits[['hit_id', 'volume_id']].rename({'hit_id':'next_hit'},axis=1),\
#                       on='next_hit', how='left').rename({'volume_id_x': 'volume_id', 'volume_id_y':'target'},axis=1)
    truth_hits = pd.merge(truth_hits, truth_hits[['hit_id', 'block']].rename({'hit_id':'next_hit'},axis=1),\
                      on='next_hit', how='left').rename({'block_x': 'block', 'block_y':'target'},axis=1)

    cell_cols = list(cell_stats.columns.values[1:])
    geom_cols = ['x', 'y', 'z', 'volume_id', 'layer_id', 'module_id', 'dist', 'rad_dist', 'block']
    
    
    X = truth_hits[geom_cols + cell_cols + ['target']]
    X.dropna(inplace=True)
    del X['block']

    
#     scaler = StandardScaler()
#     scaler.fit(X.drop(['target'],axis=1))

#     X_scaled = scaler.transform(X.drop(['target'],axis=1))

    
    
#     adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

#     model.compile(loss='categorical_crossentropy',
#                   metrics=['acc'],
#                 optimizer=adam)
#     model.load_weights('nn.h5')
    
    
#     model.fit(x=X_scaled,
#           y=to_categorical(pd.factorize(X['target'])[0]),
#           batch_size=20000,
#           epochs=10000,
#           verbose=1,
#           callbacks=[model_checkpoint],
#           validation_split=0.7)


    if cnt == 0 or cnt == 1:
        X_val = pd.concat([X_val, X], axis=0)
    else:
        X_train = pd.concat([X_train, X], axis=0)
    
    print(cnt)

    
    
    cnt += 1

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [73]:
blocks = truth_hits.groupby('block')['particle_id'].count().reset_index()[['block']].values.reshape(48)
block_stat = pd.DataFrame(np.zeros((48, 48)))
block_stat.index = blocks
block_stat.columns = blocks

tmp = truth_hits.groupby('particle_id')['block'].apply(lambda x: x.tolist()).reset_index()
tmp['block'].apply(lambda x: proc_arr(x))
1

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if sys.path[0] == '':


1

In [70]:
X_train.shape, X_val.shape

((1765269, 15), (134030, 15))

In [71]:
X_train

,x,y,z,volume_id,layer_id,module_id,dist,rad_dist,value_mean,value_max,value_min,value_sum,value_std,value_count,target
0,9.364960,30.961100,-70.763000,8,2,92,6.053694e+03,1046.292160,0.051155,0.069791,0.003011,0.767318,0.020605,15,8_4
1,20.655001,69.188499,-159.526993,8,4,151,3.066254e+04,5213.677510,0.050154,0.066363,0.002249,0.702157,0.019246,14,8_6
2,32.586601,110.810997,-255.931000,8,6,193,7.884164e+04,13340.963640,0.051876,0.067448,0.019238,0.726261,0.015232,14,8_8
3,47.328098,164.212006,-379.320007,8,8,133,1.730892e+05,29205.531677,0.053440,0.075336,0.001484,0.748162,0.018845,14,8_8
4,47.827301,166.072998,-383.605011,8,8,134,1.770205e+05,29867.691404,0.050169,0.070216,0.015011,0.752539,0.019248,15,13_2
5,69.783600,250.727997,-578.400024,13,2,189,4.022809e+05,67734.279201,1.000000,1.000000,1.000000,2.000000,0.000000,2,13_4
6,93.669403,350.313995,-806.000000,13,4,152,7.811299e+05,131493.852419,1.000000,1.000000,1.000000,2.000000,0.000000,2,12_12
9,174.895996,788.224976,-1797.500000,16,8,71,3.882893e+06,651887.221587,1.000000,1.000000,1.000000,2.000000,0.000000,2,16_6
11,12.159400,29.589500,-56.582199,8,2,107,4.224935e+03,1023.389544,0.052370,0.079143,0.008670,0.628443,0.023846,12,8_4
12,27.240499,65.687103,-127.304001,8,4,182,2.126315e+04,5056.840349,0.051301,0.071653,0.012158,0.615615,0.019526,12,8_4


In [76]:
block_stat['block'] = block_stat.index.values

In [77]:
block_stat

,12_10,12_12,12_2,12_4,12_6,12_8,13_2,13_4,13_6,13_8,...,8_6,8_8,9_10,9_12,9_14,9_2,9_4,9_6,9_8,block
12_10,111.0,14.0,0.0,0.0,5.0,667.0,3.0,9.0,15.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12_10
12_12,694.0,109.0,0.0,1.0,1.0,9.0,2.0,7.0,21.0,11.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12_12
12_2,0.0,0.0,90.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12_2
12_4,1.0,0.0,619.0,92.0,4.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12_4
12_6,2.0,1.0,8.0,627.0,93.0,2.0,1.0,1.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12_6
12_8,5.0,2.0,0.0,5.0,646.0,102.0,0.0,1.0,2.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12_8
13_2,11.0,257.0,0.0,1.0,3.0,2.0,987.0,3769.0,27.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13_2
13_4,18.0,298.0,0.0,0.0,1.0,3.0,0.0,942.0,3139.0,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13_4
13_6,14.0,205.0,0.0,0.0,0.0,6.0,0.0,0.0,793.0,2413.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13_6
13_8,0.0,8.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,606.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13_8


In [78]:
X_train['block'] = X_train['volume_id'].astype(str) + "_" + X_train['layer_id'].astype(str)
X_val['block'] = X_val['volume_id'].astype(str) + "_" + X_val['layer_id'].astype(str)

X_train = pd.merge(X_train, block_stat, on='block', how='left')
X_val = pd.merge(X_val, block_stat, on='block', how='left')

del X_train['block']
del X_val['block']

In [83]:
#%%time
import lightgbm as lgbm
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

scores = []

cat = ['module_id', 'volume_id', 'layer_id']

lgb = lgbm.sklearn.LGBMClassifier()
lgb.fit(X_train.drop(['target'] +cat ,axis=1), X_train['target'])

pred = lgb.predict(X_val.drop(['target']+cat,axis=1))#[:,1]


print(accuracy_score( X_val['target'], pred))

0.8855703946877564


/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [66]:
pred[:20]

array(['8_6', '8_8', '13_2', '13_4', '13_4', '13_8', '18_2', '8_2', '8_4',
       '8_6', '8_6', '8_8', '8_8', '13_2', '13_4', '13_6', '17_2', '17_4',
       '8_2', '8_4'], dtype=object)

In [67]:
X_val['target'].values[:20]

array(['8_6', '8_8', '13_2', '13_4', '13_4', '13_8', '18_2', '8_2', '8_4',
       '8_6', '8_6', '8_8', '8_8', '13_2', '13_4', '13_6', '17_2', '17_4',
       '8_2', '8_4'], dtype=object)

In [ ]:
0.8853540252182347

In [ ]:
0.9568529433708871
0.9566738789823174

In [86]:
imp = pd.DataFrame()
imp['feature'] = X_train.drop(['target'], axis=1).columns.values
imp['importance'] = lgb.feature_importances_
imp = imp.sort_values('importance', ascending=False)
imp.index = range(len(imp))

ValueError: Length of values does not match length of index

In [31]:
imp

,feature,importance
0,rad_dist,5439
1,z,3702
2,dist,3103
3,value_sum,2746
4,x,2536
5,y,2468
6,module_id,2114
7,value_mean,1050
8,value_min,795
9,value_count,783


In [39]:
X.columns.values

array(['x', 'y', 'z', 'volume_id', 'layer_id', 'module_id', 'dist',
       'rad_dist', 'value_mean', 'value_max', 'value_min', 'value_sum',
       'value_std', 'value_count', 'target', 'fold'], dtype=object)

In [36]:
pd.get_dummies(X_train, columns=['volume_id', 'layer_id'])

,x,y,z,module_id,dist,rad_dist,value_mean,value_max,value_min,value_sum,...,volume_id_16,volume_id_17,volume_id_18,layer_id_2,layer_id_4,layer_id_6,layer_id_8,layer_id_10,layer_id_12,layer_id_14
0,9.364960,30.961100,-70.763000,92,6.053694e+03,1046.292160,0.051155,0.069791,0.003011,0.767318,...,0,0,0,1,0,0,0,0,0,0
1,20.655001,69.188499,-159.526993,151,3.066254e+04,5213.677510,0.050154,0.066363,0.002249,0.702157,...,0,0,0,0,1,0,0,0,0,0
2,32.586601,110.810997,-255.931000,193,7.884164e+04,13340.963640,0.051876,0.067448,0.019238,0.726261,...,0,0,0,0,0,1,0,0,0,0
3,47.328098,164.212006,-379.320007,133,1.730892e+05,29205.531677,0.053440,0.075336,0.001484,0.748162,...,0,0,0,0,0,0,1,0,0,0
4,47.827301,166.072998,-383.605011,134,1.770205e+05,29867.691404,0.050169,0.070216,0.015011,0.752539,...,0,0,0,0,0,0,1,0,0,0
5,69.783600,250.727997,-578.400024,189,4.022809e+05,67734.279201,1.000000,1.000000,1.000000,2.000000,...,0,0,0,1,0,0,0,0,0,0
6,93.669403,350.313995,-806.000000,152,7.811299e+05,131493.852419,1.000000,1.000000,1.000000,2.000000,...,0,0,0,0,1,0,0,0,0,0
9,174.895996,788.224976,-1797.500000,71,3.882893e+06,651887.221587,1.000000,1.000000,1.000000,2.000000,...,1,0,0,0,0,0,1,0,0,0
11,12.159400,29.589500,-56.582199,107,4.224935e+03,1023.389544,0.052370,0.079143,0.008670,0.628443,...,0,0,0,1,0,0,0,0,0,0
12,27.240499,65.687103,-127.304001,182,2.126315e+04,5056.840349,0.051301,0.071653,0.012158,0.615615,...,0,0,0,0,1,0,0,0,0,0


In [38]:
pd.get_dummies(X_train.drop(['target'],axis=1), columns=['volume_id', 'layer_id']  ) 

,x,y,z,module_id,dist,rad_dist,value_mean,value_max,value_min,value_sum,...,volume_id_16,volume_id_17,volume_id_18,layer_id_2,layer_id_4,layer_id_6,layer_id_8,layer_id_10,layer_id_12,layer_id_14
0,9.364960,30.961100,-70.763000,92,6.053694e+03,1046.292160,0.051155,0.069791,0.003011,0.767318,...,0,0,0,1,0,0,0,0,0,0
1,20.655001,69.188499,-159.526993,151,3.066254e+04,5213.677510,0.050154,0.066363,0.002249,0.702157,...,0,0,0,0,1,0,0,0,0,0
2,32.586601,110.810997,-255.931000,193,7.884164e+04,13340.963640,0.051876,0.067448,0.019238,0.726261,...,0,0,0,0,0,1,0,0,0,0
3,47.328098,164.212006,-379.320007,133,1.730892e+05,29205.531677,0.053440,0.075336,0.001484,0.748162,...,0,0,0,0,0,0,1,0,0,0
4,47.827301,166.072998,-383.605011,134,1.770205e+05,29867.691404,0.050169,0.070216,0.015011,0.752539,...,0,0,0,0,0,0,1,0,0,0
5,69.783600,250.727997,-578.400024,189,4.022809e+05,67734.279201,1.000000,1.000000,1.000000,2.000000,...,0,0,0,1,0,0,0,0,0,0
6,93.669403,350.313995,-806.000000,152,7.811299e+05,131493.852419,1.000000,1.000000,1.000000,2.000000,...,0,0,0,0,1,0,0,0,0,0
9,174.895996,788.224976,-1797.500000,71,3.882893e+06,651887.221587,1.000000,1.000000,1.000000,2.000000,...,1,0,0,0,0,0,1,0,0,0
11,12.159400,29.589500,-56.582199,107,4.224935e+03,1023.389544,0.052370,0.079143,0.008670,0.628443,...,0,0,0,1,0,0,0,0,0,0
12,27.240499,65.687103,-127.304001,182,2.126315e+04,5056.840349,0.051301,0.071653,0.012158,0.615615,...,0,0,0,0,1,0,0,0,0,0


In [39]:
scaler = StandardScaler()
scaler.fit(pd.get_dummies(X_train.drop(['target'],axis=1), columns=['volume_id', 'layer_id']  ) )

X_train_scaled = scaler.transform(pd.get_dummies(X_train.drop(['target'],axis=1), columns=['volume_id', 'layer_id']  ) )

X_val_scaled = scaler.transform(pd.get_dummies(X_val.drop(['target'],axis=1), columns=['volume_id', 'layer_id']  ) )

In [43]:
input_1 = Input(shape=(28,), dtype='float')

x = Dense(500, activation='relu')(input_1)
x = Dropout(0.2)(x)
x = Dense(300, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(300, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(200, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(50, activation='relu')(x)
x = Dropout(0.2)(x)
preds = Dense(9, activation='softmax')(x)

model = Model(inputs=input_1, \
    outputs=preds)

adam = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(loss='categorical_crossentropy',
              metrics=['acc'],
            optimizer=adam)
    
model_checkpoint = ModelCheckpoint('nn.h5', save_best_only=True, save_weights_only=True)

In [ ]:
pd.factorize(X_train['target'])[0])

In [50]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(X_train['target'])

LabelEncoder()

In [54]:
model.fit(x=X_train_scaled,
          y=to_categorical(le.transform(X_train['target'])),
          batch_size=20000,
          epochs=1000,
          validation_data=(X_val_scaled,to_categorical(le.transform(X_val['target']))),
          verbose=1,
          callbacks=[model_checkpoint])



Train on 1765269 samples, validate on 134030 samples
Epoch 1/1000
1765269/1765269 [==============================] - 4s 2us/step - loss: 0.8093 - acc: 0.8121 - val_loss: 0.2185 - val_acc: 0.9264
Epoch 2/1000
1765269/1765269 [==============================] - 4s 2us/step - loss: 0.2507 - acc: 0.9183 - val_loss: 0.2041 - val_acc: 0.9281
Epoch 3/1000
1765269/1765269 [==============================] - 4s 2us/step - loss: 0.2334 - acc: 0.9221 - val_loss: 0.2002 - val_acc: 0.9294
Epoch 4/1000
1765269/1765269 [==============================] - 4s 2us/step - loss: 0.2260 - acc: 0.9239 - val_loss: 0.1981 - val_acc: 0.9294
Epoch 5/1000
1765269/1765269 [==============================] - 4s 2us/step - loss: 0.2213 - acc: 0.9247 - val_loss: 0.1964 - val_acc: 0.9295
Epoch 6/1000
1765269/1765269 [==============================] - 4s 2us/step - loss: 0.2182 - acc: 0.9254 - val_loss: 0.1955 - val_acc: 0.9297
Epoch 7/1000
1765269/1765269 [==============================] - 4s 2us/step - loss: 0.2163 - ac

Epoch 58/1000
1765269/1765269 [==============================] - 4s 2us/step - loss: 0.1921 - acc: 0.9305 - val_loss: 0.1812 - val_acc: 0.9333
Epoch 59/1000
1765269/1765269 [==============================] - 4s 2us/step - loss: 0.1922 - acc: 0.9306 - val_loss: 0.1829 - val_acc: 0.9328
Epoch 60/1000
1765269/1765269 [==============================] - 4s 2us/step - loss: 0.1919 - acc: 0.9304 - val_loss: 0.1815 - val_acc: 0.9334
Epoch 61/1000
1765269/1765269 [==============================] - 4s 2us/step - loss: 0.1912 - acc: 0.9307 - val_loss: 0.1812 - val_acc: 0.9337
Epoch 62/1000
1765269/1765269 [==============================] - 4s 2us/step - loss: 0.1911 - acc: 0.9308 - val_loss: 0.1815 - val_acc: 0.9336
Epoch 63/1000
1765269/1765269 [==============================] - 4s 2us/step - loss: 0.1914 - acc: 0.9307 - val_loss: 0.1820 - val_acc: 0.9332
Epoch 64/1000
1765269/1765269 [==============================] - 4s 2us/step - loss: 0.1911 - acc: 0.9308 - val_loss: 0.1813 - val_acc: 0.9339

KeyboardInterrupt: 

In [55]:
adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(loss='categorical_crossentropy',
              metrics=['acc'],
            optimizer=adam)
    
model_checkpoint = ModelCheckpoint('nn.h5', save_best_only=True, save_weights_only=True)

In [57]:
model.fit(x=X_train_scaled,
          y=to_categorical(le.transform(X_train['target'])),
          batch_size=20000,
          epochs=1000,
          validation_data=(X_val_scaled,to_categorical(le.transform(X_val['target']))),
          verbose=1,
          callbacks=[model_checkpoint])



Train on 1765269 samples, validate on 134030 samples
Epoch 1/1000
1765269/1765269 [==============================] - 4s 2us/step - loss: 0.8609 - acc: 0.7916 - val_loss: 0.2389 - val_acc: 0.9180
Epoch 2/1000
1765269/1765269 [==============================] - 4s 2us/step - loss: 0.2548 - acc: 0.9160 - val_loss: 0.2023 - val_acc: 0.9280
Epoch 3/1000
1765269/1765269 [==============================] - 4s 2us/step - loss: 0.2299 - acc: 0.9220 - val_loss: 0.1944 - val_acc: 0.9294
Epoch 4/1000
1765269/1765269 [==============================] - 4s 2us/step - loss: 0.2199 - acc: 0.9240 - val_loss: 0.1918 - val_acc: 0.9304
Epoch 5/1000
1765269/1765269 [==============================] - 4s 2us/step - loss: 0.2134 - acc: 0.9255 - val_loss: 0.1889 - val_acc: 0.9310
Epoch 6/1000
1765269/1765269 [==============================] - 4s 2us/step - loss: 0.2097 - acc: 0.9261 - val_loss: 0.1876 - val_acc: 0.9305
Epoch 7/1000
1765269/1765269 [==============================] - 4s 2us/step - loss: 0.2067 - ac

KeyboardInterrupt: 